### Simulate miRNA reads using mature and hairpin miRNA database 

In [1]:
from itertools import islice

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

import random

### Every read name in fastq must contain
                    miRNA and hairpin name, 
                    start and end position of miRNA in Hairpin, 
                    templated gain or loss,
                    mutations, 
                    additions.
                    Example: @hsa-let-7b_hsa-let-7b-5p_5:32_5:0_mut:1G/3G_add:null

In [2]:
# Create dataframe with database motifs, mature and hairpin miRNAs 


def readMotifsFromFile(filename):
    file=open(filename, 'r')
    cols = ["name","motif","sequence"]
    readsD = {i:[] for i in cols}
    reads=pd.DataFrame(columns=cols)
    for line in file: 
        if(line.startswith(">")): 
            name = list(filter(None, line.rstrip().split(' ')))
            readsD['name'].append('@'+name[0][1:])
            readsD['motif'].append(name[1])
        else: 
            readsD['sequence'].append(line.rstrip())      
    file.close()
    
    reads=pd.DataFrame(columns=cols)
    return reads.append(pd.DataFrame.from_dict(readsD))



def readMIRNAfromFile(filename):
    file=open(filename, 'r')
    cols = ["name","sequence","qualities"]
    readsD = {i:[] for i in cols}
    reads=pd.DataFrame(columns=cols)
    for line in file: 
        if(line.startswith(">")): 
            name = "@" + line.split(' ')[0][1:]
            readsD['name'].append(name)
        else: 
            readsD['sequence'].append(line.rstrip())
            quals = ''
            for i in range(1, len(line)):
                quals = quals + quality[int(45 - 10/(i**0.5))]
            readsD['qualities'].append(quals)

    file.close()
    reads=pd.DataFrame(columns=cols)
    return reads.append(pd.DataFrame.from_dict(readsD))


def matureInHairpin(dfH, s, left, right):
    newS = dfH[dfH.sequence.str.find(s['sequence'])!=-1].iloc[0]
    pos = newS.sequence.find(s.sequence)
    if (pos<left): 
        left = pos
        
    quals = "I" * left + s.qualities + "I" * right  
    s.sequence = newS.sequence[pos - left:pos+len(s.sequence)+right]
    s.qualities = quals[:len(s.sequence)]
    s["name"] = newS["name"] + "_" + s["name"][1:] + "_" +  str(pos) + ':' + str(pos + len(s.sequence)) + "_" + str(left) + ':' + str(right)
    return s



def dataFrameToFastq(df, filename):
    thefile = open(filename, 'w')
    for index, row in df.iterrows():
        thefile.write("%s\n" % row['name'])
        thefile.write("%s\n" % row['sequence'])
        thefile.write("%s\n" % "+")
        thefile.write("%s\n" % row['qualities'])    
    thefile.close()



In [ ]:
# Introduce variations and indels to reads, functions work with apply: 


def randomVariation(df, l):
    if(len(l)==0): 
        if(df['name'].find( "mut:")==-1):
            df['name'] = df['name'] + "_mut:null" 
        return df
    s=''
    for i in l: 
        bases = ['A','T','C','G']
        bases.remove(df.sequence[i])
        s = s + str(i) + df.sequence[i] + '/'
        if(i<0):
            df.sequence = df.sequence[:i-1] + random.choice(bases) + df.sequence[i:]
        else:
            df.sequence = df.sequence[:i] + random.choice(bases) + df.sequence[i+1:]
        
    if(s.endswith('/')): 
        s = s[:-1]
    if(df['name'].find( "mut:")==-1):
        df['name'] = df['name'] + "_mut:" + s
    else: 
        name = df['name'].split('_')
        for i in name: 
            if i.startswith('mut:'):
                i = i+'/'+s
        df['name'] = '_'.join(name)
    return df
    
def randomDeletion(df, l):
    for i in l: 
        s = str(df.sequence[:i] + df.sequence[i+1:])
        q = str(df.qualities[:i] + df.qualities[i+1:])
    df.sequence = s
    df.qualities = q
    name = df['name'].split(':')
    name[2] = name[2] + "-" + str(len(l)) + "D"
    df['name'] = ':'.join(name)
    return df


def randomInsertion(df, l):
    bases = ['A','T','C','G']
    for i in l: 
        s = str(df.sequence[:i] + random.choice(bases) + df.sequence[i+1:])
        q = str(df.qualities[:i] + 'I' + df.qualities[i+1:])
    df.sequence = s
    df.qualities = q
    name = df['name'].split(':')
    name[2] = name[2] + "-" + str(len(l)) + "I"
    df['name'] = ':'.join(name)
    return df

def cutBases(df, where):
    if(where==5):
        n = random.randint(1,2)
        #m = df['sequence'].find(df['motif'])
        #n = min(n,m)
        loss = df['sequence'][:n]
        df['sequence'] = df['sequence'][n:]
        df['qualities'] = df['qualities'][n:]
    else: 
        sLen=len(df['sequence'])
        n = sLen - random.randint(1,5)
        #m = df['sequence'].find(df['motif']) + len(df['motif'])
        #n  = max(n,m)
        loss = df['sequence'][n:]
        df['sequence'] = df['sequence'][:n]
        df['qualities'] = df['qualities'][:n]
        
    df['name'] = df['name'] + "_add:" + 'loss-' + str(where) + 'p-' + loss
    return df

def addBases(df, s, where):
    if(len(s)==0): 
        df['name'] = df['name'] + "_add:null" 
        return df
    if(where==3):
        df.sequence = df.sequence + s
        df.qualities = df.qualities + "I" * len(s)
    else: 
        df.sequence = s + df.sequence
        df.qualities = "I" * len(s) + df.qualities

    df['name'] = df['name'] + "_add:" + s
    return df

def addRandomBases(df, n, where):
    bases = ['A','T','C','G']
    if(n==0): 
        df['name'] = df['name'] + "_add:null" 
        return df
    randSeq = ''.join(np.random.choice(bases,n))
    if(where==3):
        df.sequence = df.sequence + randSeq
        df.qualities = df.qualities + "I" * n
    else: 
        df.sequence =  randSeq + df.sequence
        df.qualities = "I" * n + df.qualities

    df['name'] = df['name'] + "_add:" + "gain-" + str(where) + 'p-' + randSeq
    return df


def addPCRErr(df, prob):
    arr = [i for i in range(0,len(df['sequence'])) if random.uniform(0,1)<prob]
    if len(arr)>0:
        seq=list(df['sequence'])
        for i in arr: 
            bases = ['A','T','C','G']
            bases.remove(seq[i])
            seq[i]=random.choice(bases)
        df['sequence']=''.join(seq) 
        df['name'] = df['name'] + "_PCRerr:" + '-'.join(map(str,arr))
    return df

In [3]:
quality = """ !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~ """


In [4]:
file_mature='mature_homo.fa'
file_hairpin='hairpin_homo.fa'

In [25]:
readsM = readMIRNAfromFile(file_mature)
readsH = readMIRNAfromFile(file_hairpin)
readsH.head()

,name,qualities,sequence
0,@hsa-let-7a-1,CEGHHHIIIIIJJJJJJJJJJJJJKKKKKKKKKKKKKKKKKKKKKK...,TGGGATGAGGTAGTAGGTTGTATAGTTTTAGGGTCACACCCACCAC...
1,@hsa-let-7a-2,CEGHHHIIIIIJJJJJJJJJJJJJKKKKKKKKKKKKKKKKKKKKKK...,AGGTTGAGGTAGTAGGTTGTATAGTTTAGAATTACATCAAGGGAGA...
2,@hsa-let-7a-3,CEGHHHIIIIIJJJJJJJJJJJJJKKKKKKKKKKKKKKKKKKKKKK...,GGGTGAGGTAGTAGGTTGTATAGTTTGGGGCTCTGCCCTGCTATGG...
3,@hsa-let-7b,CEGHHHIIIIIJJJJJJJJJJJJJKKKKKKKKKKKKKKKKKKKKKK...,CGGGGTGAGGTAGTAGGTTGTGTGGTTTCAGGGCAGTGATGTTGCC...
4,@hsa-let-7c,CEGHHHIIIIIJJJJJJJJJJJJJKKKKKKKKKKKKKKKKKKKKKK...,GCATCCGGGTTGAGGTAGTAGGTTGTATGGTTTAGAGTTACACCCT...


In [27]:
# Multiply number of reads 

readsSim = readsM.copy()
readsSim = readsSim.append(readsSim).reset_index(drop=True)
readsSim = readsSim.append(readsSim).reset_index(drop=True)
readsSim = readsSim.append(readsSim).reset_index(drop=True)


In [28]:
readsSim.head()

,name,qualities,sequence
0,@hsa-let-7a-5p,CEGHHHIIIIIJJJJJJJJJJJ,TGAGGTAGTAGGTTGTATAGTT
1,@hsa-let-7a-3p,CEGHHHIIIIIJJJJJJJJJJ,CTATACAATCTACTGTCTTTC
2,@hsa-let-7a-2-3p,CEGHHHIIIIIJJJJJJJJJJJ,CTGTACAGCCTCCTAGCTTTCC
3,@hsa-let-7b-5p,CEGHHHIIIIIJJJJJJJJJJJ,TGAGGTAGTAGGTTGTGTGGTT
4,@hsa-let-7b-3p,CEGHHHIIIIIJJJJJJJJJJJ,CTATACAACCTACTGCCTTCCC


In [193]:
# Templated gain 1-2nt 3' end + pcr noise
where=3
readsSim1 = readsSim.copy()
prob=0.001

readsSim1.apply(lambda x: matureInHairpin(readsH,x,0,random.randint(1,2)), axis=1)
readsSim1.apply(lambda x: randomVariation(x, []), axis=1)
readsSim1.apply(lambda x: addBases(x, '', where), axis=1)

readsSim1.apply(lambda x: addPCRErr(x, prob), axis=1)

filename = 'test_gainTemp_' + str(where) + 'p_' + 'mirnaPCRerr.fastq'
dataFrameToFastq(readsSim1, filename)

In [194]:
# Templated gain 1-2nt 5' end + pcr noise
where=5
readsSim1 = readsSim.copy()
prob=0.001

readsSim1.apply(lambda x: matureInHairpin(readsH,x,random.randint(1,2),0), axis=1)
readsSim1.apply(lambda x: randomVariation(x, []), axis=1)
readsSim1.apply(lambda x: addBases(x, '', where), axis=1)

readsSim1.apply(lambda x: addPCRErr(x, prob), axis=1)

filename = 'test_gainTemp_' + str(where) + 'p_' + 'mirnaPCRerr.fastq'
dataFrameToFastq(readsSim1, filename)

In [195]:
# Unemplated gain 1-7nt 3' end + pcr noise
where=3
readsSim1 = readsSim.copy()
prob=0.001

readsSim1.apply(lambda x: matureInHairpin(readsH,x,0,0), axis=1)
readsSim1.apply(lambda x: randomVariation(x, []), axis=1)
readsSim1.apply(lambda x: addRandomBases(x, random.randint(1,7),where), axis=1)

readsSim1.apply(lambda x: addPCRErr(x, prob), axis=1)

filename = 'test_gainRand_' + str(where) + 'p_' + 'mirnaPCRerr.fastq'
dataFrameToFastq(readsSim1, filename)

In [196]:
# Unemplated gain 1-2nt 5' end
where=5
readsSim1 = readsSim.copy()
prob=0.001

readsSim1.apply(lambda x: matureInHairpin(readsH,x,0,0), axis=1)
readsSim1.apply(lambda x: randomVariation(x, []), axis=1)
readsSim1.apply(lambda x: addRandomBases(x, random.randint(1,2),where), axis=1)
readsSim1.apply(lambda x: addPCRErr(x, prob), axis=1)

filename = 'test_gainRand_' + str(where) + 'p_' + 'mirnaPCRerr.fastq'
dataFrameToFastq(readsSim1, filename)

# Motifs

In [29]:
motifs = readMotifsFromFile('motif-list-13-new.fa')

In [ ]:
#motifs[~motifs['name'].isin(readsM['name'])]
#readsM[readsM['name'].isin(motifs['name'])]

In [30]:
MotifsAndReads = pd.merge(readsM, motifs, how='inner', on=['name','sequence'])

In [31]:
readsSim = MotifsAndReads.copy()
readsSim = readsSim.append(readsSim).reset_index(drop=True)
readsSim = readsSim.append(readsSim).reset_index(drop=True)
readsSim = readsSim.append(readsSim).reset_index(drop=True)

In [15]:
#3' end trimming 1-2nt + pcr error
where=3
prob=0.001
readsSim1 = readsSim.copy()

readsSim1.apply(lambda x: matureInHairpin(readsH,x,0,0), axis=1)
readsSim1.apply(lambda x: randomVariation(x, []), axis=1)
readsSim1.apply(lambda x: cutBases(x,where),axis=1)

readsSim1.apply(lambda x: addPCRErr(x, prob), axis=1)

filename = 'test_lossRand_' + str(where) + 'p_' + 'mirnaPCRerr.fastq'
dataFrameToFastq(readsSim1, filename)

In [201]:
#5' end trimming 1-2nt + pcr error
where=5
prob=0.001
readsSim1 = readsSim.copy()

readsSim1.apply(lambda x: matureInHairpin(readsH,x,0,0), axis=1)
readsSim1.apply(lambda x: randomVariation(x, []), axis=1)
readsSim1.apply(lambda x: cutBases(x,where),axis=1)

readsSim1.apply(lambda x: addPCRErr(x, prob), axis=1)


filename = 'test_lossRand_' + str(where) + 'p_' + 'mirnaPCRerr.fastq'
dataFrameToFastq(readsSim1, filename)

In [202]:
#5' end SNiPs (but not in motifs) + pcr error
where=5
prob=0.001
readsSim1 = readsSim.copy()

readsSim1.apply(lambda x: matureInHairpin(readsH,x,0,0), axis=1)
readsSim1.apply(lambda x: randomVariation(x, [random.randint(0,x['sequence'].find(x['motif'])-1)]), 
                          axis=1)

readsSim1.apply(lambda x: addBases(x, '', where), axis=1)

readsSim1.apply(lambda x: addPCRErr(x, prob), axis=1)


filename = 'test_mirnaSNiP_' + str(where) + 'p_' + 'mirnaPCRerr.fastq'
dataFrameToFastq(readsSim1, filename)

##Check if any snip is in the motif 
#any(readsSim1.apply(lambda x:  x['sequence'].find(x['motif']) - int(x['name'].split('_')[4][4])<0, axis=1 ) )


In [205]:
#3' end SNiPs (but not in motifs) + pcr error
where=3
prob=0.001
readsSim1 = readsSim.copy()

readsSim1.apply(lambda x: matureInHairpin(readsH,x,0,0), axis=1)
readsSim1.apply(lambda x: randomVariation(x, [random.randint(x['sequence'].find(x['motif'])+len(x['motif']),
                                                          len(x['sequence'])-1)]),
                axis=1)

readsSim1.apply(lambda x: addBases(x, '', where), axis=1)

readsSim1.apply(lambda x: addPCRErr(x, prob), axis=1)


filename = 'test_mirnaSNiP_' + str(where) + 'p_' + 'mirnaPCRerr.fastq'
dataFrameToFastq(readsSim1, filename)


In [35]:
#SNiP everywhere + SNip & pcr error

prob=0.001
readsSim1 = readsSim.copy()

readsSim1.apply(lambda x: matureInHairpin(readsH,x,0,0), axis=1)
readsSim1.apply(lambda x: randomVariation(x, [random.randint(0,len(x['sequence'])-1)]), axis=1)         
readsSim1.apply(lambda x: addBases(x, '', 3), axis=1)

filename = 'test_SNiPeverywhere.fastq'
dataFrameToFastq(readsSim1, filename)

readsSim1.apply(lambda x: addPCRErr(x, prob), axis=1)

filename = 'test_SNiPeverywhere_PCRerr.fastq'
dataFrameToFastq(readsSim1, filename)


In [175]:
#PCR error only 
prob=0.001
readsSim1 = readsSim.copy()

readsSim1.apply(lambda x: matureInHairpin(readsH,x,0,0), axis=1)
readsSim1.apply(lambda x: addBases(x, '', 3), axis=1)
readsSim1.apply(lambda x: randomVariation(x, []), axis=1)

readsSim1.apply(lambda x: addPCRErr(x, prob), axis=1)


filename = 'test_mirnaPCRerr.fastq'
dataFrameToFastq(readsSim1, filename)

,name,qualities,sequence
0,@hsa-let-7a-1_hsa-let-7a-1_hsa-let-7a-5p_5:27_...,CEGHHHIIIIIJJJJJJJJJJJ,TGAGGTAGTAGGTTGTATAGTT
1,@hsa-let-7a-1_hsa-let-7a-1_hsa-let-7a-3p_56:77...,CEGHHHIIIIIJJJJJJJJJJ,CTATACAATCTACTGTCTTTC
2,@hsa-let-7a-2_hsa-let-7a-2_hsa-let-7a-2-3p_49:...,CEGHHHIIIIIJJJJJJJJJJJ,CTGTACAGCCTCCTAGCTTTCC
3,@hsa-let-7b_hsa-let-7b_hsa-let-7b-5p_5:27_0:0_...,CEGHHHIIIIIJJJJJJJJJJJ,TGAGGTAGTAGGTTGTGTGGTT
4,@hsa-let-7b_hsa-let-7b_hsa-let-7b-3p_59:81_0:0...,CEGHHHIIIIIJJJJJJJJJJJ,CTATACAACCTACTGCCTTCCC
5,@hsa-let-7c_hsa-let-7c_hsa-let-7c-5p_10:32_0:0...,CEGHHHIIIIIJJJJJJJJJJJ,TGAGGTAGTAGGTTGTATGGTT
6,@hsa-let-7c_hsa-let-7c_hsa-let-7c-3p_55:77_0:0...,CEGHHHIIIIIJJJJJJJJJJJ,CTGTACAACCTTCTAGCTTTCC
7,@hsa-let-7d_hsa-let-7d_hsa-let-7d-5p_7:29_0:0_...,CEGHHHIIIIIJJJJJJJJJJJ,AGAGGTAGTAGGTTGCATAGTT
8,@hsa-let-7d_hsa-let-7d_hsa-let-7d-3p_61:83_0:0...,CEGHHHIIIIIJJJJJJJJJJJ,CTATACGACCTGCTGCCTTTCT
9,@hsa-let-7e_hsa-let-7e_hsa-let-7e-5p_7:29_0:0_...,CEGHHHIIIIIJJJJJJJJJJJ,TGAGGTAGGAGGTTGTATAGTT


In [203]:
#Number of reads with PCR error

len(readsSim1[readsSim1['name'].str.contains("_PCRerr")])

383

In [210]:
for i, val in readsSim1.iterrows(): 
    if(readsSim['sequence'].loc[i].startswith(val['sequence'])):
        print('not true')